# Chapter 10: Sequence Hacking, Hashing and Slicing

This chapter generalises the `Vector2d` class from the previous chapter to the multidimensional case.

`Vector` will behave as a standard Python immutable flat sequence with elements that are floats. By the end of this chapter it will support:

* Basic sequence protocol: `__len__`, `__getitem__`.

* Safe representation of many instances with many items.

* Proper slicing support, producing new `Vector` instances.

* Aggregate hashing taking into account every contained element value.

* Custom formatting language extension.

`Vector` will also implement dynamic attribute access with `__getattr__` as a way of replacing read-only properties of `Vector2d` (not typical of sequence types).


## Vector: A User-Defined Sequence Type

The strategy to implement `Vector` will be to use composition instead of inheritance. The components will be stored in `array` of floats, and the necessary methods will be implemented to make `Vector` behave like an immutable flat sequence (an example of *duck-typing*, which we will return to).

### Vector Take #1: Vector2d Compatible

Best practice for a sequence constructor is to take the data as an iterable argument in the constructor, like all built-in sequence types do.

Note the use of `reprlib` to condense the output of  `__repr__`. This is because multi-dimensional vectors can be arbitrarily high-dimensional, and we want to control the output somehow.

In [23]:
# Example 10-2. vector_v1.py: derived from vector2d_v1.py

from array import array
import reprlib
import math

class Vector:
    typecode = "d"
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        # Get limited-length representation of self._components
        components = components[components.find("["):-1]
        class_name = type(self).__name__
        return f"{class_name}({components})"
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])) + bytes(self._components)
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    

**Implementation detail:** In `__repr__`, the same result could have been achieved by `reprlib.repr(list(self._components))`. This would be wasteful because it would require copying every item from `self._components` to a `list` just to use the `list` `repr`.

## Protocols and Duck Typing

TODO.